In [19]:
import pandas as pd
import numpy as np

### Распакуем данные в отдельную папку

In [3]:
import os
import zipfile

# Path to the ZIP file
zip_filepath = './hse-rec-sys-challenge-2024.zip'

# Destination folder where the extracted files will be placed
destination_folder = 'data'

# Ensure that the destination folder exists
os.makedirs(destination_folder, exist_ok=True)

# Unzip the file to the specified folder
with zipfile.ZipFile(zip_filepath, 'r') as zf:
    zf.extractall(path=destination_folder)

### Посмотрим на содержимое

In [ ]:
events_df = pd.read_csv('./data/events.csv')
events_df.head()

In [ ]:
events_df[events_df['user_id']==0]

In [ ]:
item_features_df = pd.read_csv('./data/item_features.csv')
item_features_df.head()

In [ ]:
user_features_df = pd.read_csv('./data/user_features.csv')
user_features_df.head()

In [ ]:
submission_sample_df = pd.read_csv('./data/submission_sample.csv')
submission_sample_df.head()

In [ ]:
submission_sample_df.item_id

### Разобьем выборку на тренировочную, валидационнную и тестовую

In [ ]:
import pandas as pd

def split_data_by_user(df, test_size=1, val_size=1):
    """
    Функция разделения данных на обучающую, валидационную и тестовую выборки.
    
    :param df: DataFrame с данными
    :param test_size: размер тестового набора (по умолчанию 1)
    :param val_size: размер валидационного набора (по умолчанию 1)
    :return: три DataFrame: train_df, val_df, test_df
    """
    # Группируем данные по каждому пользователю
    grouped = df.groupby('user_id')
    
    # Список для хранения индексов каждой группы
    train_indices = []
    val_indices = []
    test_indices = []
    
    # Проходимся по каждой группе
    for _, group in grouped:
        # Сортируем группу по timestamp
        sorted_group = group.sort_values(by='timestamp', ascending=False)
        
        # Получаем индексы для каждой выборки
        test_idx = sorted_group.index[:test_size]
        val_idx = sorted_group.index[test_size:test_size + val_size]
        train_idx = sorted_group.index[test_size + val_size:]
        
        # Добавляем индексы в соответствующие списки
        train_indices.extend(train_idx)
        val_indices.extend(val_idx)
        test_indices.extend(test_idx)
    
    # Формируем DataFrames для каждой выборки
    train_df = df.loc[train_indices].copy()
    val_df = df.loc[val_indices].copy()
    test_df = df.loc[test_indices].copy()
    
    return train_df, val_df, test_df

# Пример использования функции
train_df, val_df, test_df = split_data_by_user(events_df)

print("Train shape:", train_df.shape)
print("Val shape:", val_df.shape)
print("Test shape:", test_df.shape)

In [ ]:
train_df.head()

### Добавляем фичи из двух других таблиц

In [32]:
def add_features_to_train_data(df, user_features_df, item_features_df):
    # Слияние с таблицей признаков пользователей
    merged_with_users = pd.merge(
        left=df,
        right=user_features_df,
        on="user_id",
        how="left"
    )
    
    # Слияние с таблицей признаков фильмов
    final_merged = pd.merge(
        left=merged_with_users,
        right=item_features_df,
        on="item_id",
        how="left"
    )
    
    return final_merged

In [ ]:
featured_train_df = add_features_to_train_data(train_df, user_features_df, item_features_df)
featured_val_df = add_features_to_train_data(val_df, user_features_df, item_features_df)
featured_test_df = add_features_to_train_data(test_df, user_features_df, item_features_df)

featured_train_df.head()

### Сформируем матрицы 

размера (n_users, n_items) для обучающего и тестового наборов таким образом, чтобы элемент в ячейке [i, j] отражал оценку i-го пользователя j-му фильму 

https://makesomecode.me/2018/09/movie-recommendation-system/

In [ ]:
n_users = user_features_df['user_id'].nunique()
n_items = item_features_df['item_id'].nunique()

n_users, n_items

In [21]:
data_matrix = np.zeros((n_users, n_items))
for line in events_df.itertuples():
    data_matrix[line[1], line[2]] = line[3]
    

In [ ]:
data_matrix

In [25]:
from  sklearn.metrics.pairwise import pairwise_distances

# считаем косинусное расстояние для пользователей и фильмов 

# (построчно и поколоночно соотвественно).

user_similarity = pairwise_distances(data_matrix, metric='cosine')
item_similarity = pairwise_distances(data_matrix.T, metric='cosine')